# Homework: Decipherment

In [1]:
%load_ext autoreload
%autoreload 2
from collections import defaultdict, Counter
import collections
import pprint
import math
import bz2
from ngram import *
import sys, string
pp = pprint.PrettyPrinter(width=45, compact=True)

First let us read in the cipher text from the `data` directory:

In [2]:
def read_file(filename):
    if filename[-4:] == ".bz2":
        with bz2.open(filename, 'rt', encoding='utf8') as f:
            content = f.read()
            f.close()
    else:
        with open(filename, 'r', encoding='utf8') as f:
            content = f.read()
            f.close()
    return content

cipher = read_file("data/cipher.txt")
print(cipher)

º∫P/Z/uB∫ÀOR•–X•B
WV+≈GyF∞ºHPπKÇ—y≈
MJy^uIÀΩ—T‘NQyDµ£
S¢/º∑BPORAu∫∆RÃ—E
À^LMZJƒ“\–FHVW≈æy
π+—GDºKI£∞—Xæµ§S¢
RN‘IyEÃOæ—GBTQS∑B
Lƒ/P∑BπX—EHMu^RRÀ
√ZK—–I£W—ÇæµLM“º∑
BPDR+j•∞\N¢≈EuHÀF
Z√–OVWIµ+‘L£Ã^R∞H
IºDR∏Ty“\ƒ≈/πXJQA
PµMæRu‘∫L£NVEKH•G
“IÇJÀµºæLMÃNA£Z¢P
§u–ÀAº∑BVW\+VT‘OP
^•S“Ã∆u≈∞ΩD§G∫∫IM
NÀ£S√E/º∫∫Z∆AP∑BV
–≈X—W—∏F∑æ√+πºAºB
∫OTµRu√+∏ƒy—∏^S—W
VZ≈GyKE∏TyAº∫∑L‘∏
HÇFBXº§XADƒ\ΩLÇ•—
∏≈ƒ∑∑∞≈µPORXQF∫G√
ZπJT‘—∏æJI+“BPQW∞
VEX“ºWI∞—EHM£•uIÀ


## Default Solution

For the default solution we need to compute statistics like length, number of symbols/letters, 
unique occurences, frequencies and relative frequencies of a given file. This is done in the function `get_statistics` below.

While using `get_statistics`, make sure that `cipher=True` is set when the input is a ciphertext.

In [3]:
def get_statistics(content, cipher=True):
    stats = {}
    content = list(content)
    split_content = [x for x in content if x != '\n' and x!=' ']
    length = len(split_content)
    symbols = set(split_content)
    uniq_sym = len(list(symbols))
    freq = collections.Counter(split_content)
    rel_freq = {}
    for sym, frequency in freq.items():
        rel_freq[sym] = (frequency/length)*100
        
    if cipher:
        stats = {'content':split_content, 'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    else:
        stats = {'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    return stats

In [4]:
cipher_desc = get_statistics(cipher, cipher=True)
# pp.pprint(cipher_desc)

The default solution matches the frequency of symbols in the cipher text with frequency of letters in the plaintext language (in this case, English). Note that this is just some text in English used to compute letter frequencies. We do not have access to the real plaintext in this homework. 

In order to do compute plaintext frequencies, we use an English dataset has no punctuation or spaces and all characters are lowercase.

In [ ]:
# plaintext description
plaintxt = read_file("data/default.wiki.txt.bz2")
plaintxt_desc = get_statistics(plaintxt, cipher=False)
# pp.pprint(plaintxt_desc)

We have all the tools we need to describe the default solution to this homework.

We use a simple frequency matching heuristic to map cipher symbols to English letters.

We match the frequencies using the function $f(\cdot)$ of each cipher symbol $c$ with each English letter $e$:

$$h_{c,e} = | \log(\frac{f(c)}{f(e)})) | $$

For each cipher text symbol $c$ we then compute the most likely plain text symbol $e$ by sorting based on the above score.

In [ ]:
"""
default : frequency matching heuristic

Notice how the candidate mappings, a.k.a hypotheses, are first scored with a measure of quality and, 
then, the best scoring hypothesis is chosen as the winner. 

The plaintext letters from the winner are then mapped to the respective ciphertext symbols.
"""

# def find_mappings(ciphertext, plaintext):
#     mappings = defaultdict(dict)
#     hypotheses = defaultdict(dict)
#     # calculate alignment scores
#     for symbol in ciphertext['vocab']:
#         for letter in plaintext['vocab']:
#             hypotheses[symbol][letter] = abs(math.log((ciphertext['relative_freq'][symbol]/plaintext['relative_freq'][letter])))
    
#     # find winner
#     for sym in hypotheses.keys():
#         #mappings[sym] = min(lemma_alignment[sym], key=lemma_alignment[sym].get)
#         winner = sorted(hypotheses[sym].items(), key=lambda kv: kv[1])
#         mappings[sym] = winner[1][0]
    
#     return mappings

Using this scoring function we map the cipher symbol `∆` to `v` in English

In [ ]:
# mapping = find_mappings(cipher_desc, plaintxt_desc)
# print("∆ maps to {}\n".format(mapping['∆']))
# print(mapping)

The default solution to this decipherment problem is to take each cipher symbol and map it to the most likely English letter as provided by the `find_mappings` function above.

In [ ]:
# english_text = []
# for symbol in cipher_desc['content']:
#     english_text.append(mapping[symbol])
# decipherment = ('').join(english_text)
# print(decipherment)

## Baseline

In [5]:
%%time
sequence = 'In a few cases, a multilingual artifact has been necessary to facilitate decipherment, the Rosetta Stone being the classic example. Statistical techniques provide another pathway to decipherment, as does the analysis of modern languages derived from ancient languages in which undeciphered texts are written. Archaeological and historical information is helpful in verifying hypothesized decipherments.'

# lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=True)
lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=False)

Reading language model from data/6-gram-wiki-char.lm.bz2...



Wall time: 1min 21s


In [6]:
print(sequence)
lm_logprob = lm.score_seq(sequence)
print("TOTAL LM LOGPROB: {}".format(lm_logprob), file=sys.stderr)

print("TOTAL LM LOGPROB: {}".format(lm.score_seq('this is the text.')), file=sys.stderr)
print("TOTAL LM LOGPROB: {}".format(lm.score_seq('jasbklfhthejkldhf')), file=sys.stderr)

print(lm.get_bitstring_spans('..oo...ooo..'))
print(lm.score_bitstring('thisisatest', 'oo...oo.ooo'))

In a few cases, a multilingual artifact has been necessary to facilitate decipherment, the Rosetta Stone being the classic example. Statistical techniques provide another pathway to decipherment, as does the analysis of modern languages derived from ancient languages in which undeciphered texts are written. Archaeological and historical information is helpful in verifying hypothesized decipherments.
{2: 3, 3: 4, 7: 8, 8: 9, 9: 10}
-8.10905897


TOTAL LM LOGPROB: -221.09434842188
TOTAL LM LOGPROB: -9.76947916
TOTAL LM LOGPROB: -40.57683077


In [7]:
def reverse_mapping(reversed_mapping):
    mapping = dict()
    for key, values in reversed_mapping.items():
        for value in values:
            mapping[value] = key
    return mapping

In [57]:
def score(phi, cipher, lm):
    mapping = reverse_mapping(phi)
    partial_cipher = ''
    bit_string = ''
    for char in cipher:
        if char in mapping.keys():
            partial_cipher += mapping[char]
            bit_string += 'o'
        else:
            partial_cipher += char
            bit_string += '_'
    score = lm.score_bitstring(partial_cipher, bit_string)
    return score

In [18]:
# test the two functions above
reversed_mapping = {'i': [cipher[0]], 'l': [cipher[1]]}
reversed_mapping

{'i': ['º'], 'l': ['∫']}

In [19]:
reverse_mapping(reversed_mapping)

{'º': 'i', '∫': 'l'}

In [20]:
score(reversed_mapping, cipher, lm)

-32.930710299999994

In [86]:
def beam_search(ext_order, ext_limits=1, topn=1):
    '''
    finds the mappings between cipher char and plaintext char, returns the mapping dictionary
    ext_order -- list, the unigram char list sorted by their count DESC
    ext_limits -- int, defines maximum number of cipher char can be mapped to a plaintext char
    topn -- int, defines the number of dictionaries we want to keep while pruning
    '''
    print('Number of unique symbols in cipher', len(ext_order))
    # mapping relationships already found with score
    Hs = [(defaultdict(dict), 0)]
    # hypothesis mapping relationships with score
    Ht = [(defaultdict(dict), 0)]
    # initialize the cardinality (number of unique cipher text)
    cardinality = 0
    # list of plaintext characters
    Ve = [chr(i) for i in range(97, 123, 1)]
    while cardinality < len(ext_order):
        f = ext_order[cardinality]
        print('\tWorking on symbol: ', f)
        for phi, _ in Hs:
            for e in Ve:
                phi_prime = phi.copy()
                if e in phi_prime.keys():
                    if len(phi_prime[e]) < ext_limits:
                        phi_prime[e].append(f)
                        Ht.append((phi_prime, score(phi_prime, cipher, lm)))
                else:
                    phi_prime[e] = [f]
                    this_score = score(phi_prime, cipher, lm)
                    Ht.append((phi_prime, this_score))
        # prune the histogram
        Ht = sorted(Ht, key=lambda x:x[1], reverse=True)[:topn]
        cardinality += 1
        Hs = Ht.copy()
        Ht.clear()
        print(Hs)
    return sorted(Hs, key=lambda x:x[1], reverse=True)

In [87]:
# cipher = read_file("data/cipher.txt")
# cipher_desc = get_statistics(cipher, cipher=True)
# plaintxt = read_file("data/default.wiki.txt.bz2")
# plaintxt_desc = get_statistics(plaintxt, cipher=False)

In [88]:
reversed_mapping = beam_search(list(cipher_desc['frequencies'].keys()), 2, 2)
#mapping = reverse_mapping(reversed_mapping)

Number of unique symbols in cipher 54
	Working on symbol:  º
[(defaultdict(<class 'dict'>, {}), 0), (defaultdict(<class 'dict'>, {'e': ['º']}), -16.009092599999995)]
	Working on symbol:  ∫
[(defaultdict(<class 'dict'>, {'e': ['∫']}), -13.7693778), (defaultdict(<class 'dict'>, {'t': ['∫']}), -14.719731)]
	Working on symbol:  P
[(defaultdict(<class 'dict'>, {'e': ['∫', 'P']}), -24.417077799999994), (defaultdict(<class 'dict'>, {'t': ['∫', 'P'], 'e': ['P']}), -24.7838144)]
	Working on symbol:  /
[(defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/']}), -30.784666699999992), (defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 't': ['/']}), -30.792450999999993)]
	Working on symbol:  Z
[(defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z']}), -38.63049360000001), (defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 't': ['/', 'Z'], 'a': ['Z']}), -39.43234700000001)]
	Working on symbol:  u
[(defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z', 'u'], 't': ['u

[(defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z', 'u'], 't': ['u', 'B'], 's': ['À', 'O'], 'i': ['R', '•'], 'r': ['•', '–'], 'h': ['–', 'X'], 'o': ['X', 'W'], 'c': ['V', '+'], 'u': ['≈', 'G'], 'l': ['G', 'y'], 'y': ['F', '∞'], 'd': ['∞', 'H'], 'f': ['π', 'K']}), -257.38378660999985), (defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z', 'u'], 't': ['u', 'B'], 's': ['À', 'O'], 'i': ['R', '•'], 'r': ['•', '–'], 'h': ['–', 'X'], 'o': ['X', 'W'], 'c': ['V', '+'], 'u': ['≈', 'G'], 'l': ['G', 'y'], 'y': ['F', '∞'], 'd': ['∞', 'H'], 'm': ['π', 'K']}), -258.8885289099999)]
	Working on symbol:  Ç
[(defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z', 'u'], 't': ['u', 'B'], 's': ['À', 'O'], 'i': ['R', '•'], 'r': ['•', '–'], 'h': ['–', 'X'], 'o': ['X', 'W'], 'c': ['V', '+'], 'u': ['≈', 'G'], 'l': ['G', 'y'], 'y': ['F', '∞'], 'd': ['∞', 'H'], 'f': ['π', 'K'], 'b': ['Ç']}), -266.25680580999983), (defaultdict(<class 'dict'>, {'e': ['

[(defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z', 'u'], 't': ['u', 'B'], 's': ['À', 'O'], 'i': ['R', '•'], 'r': ['•', '–'], 'h': ['–', 'X'], 'o': ['X', 'W'], 'c': ['V', '+'], 'u': ['≈', 'G'], 'l': ['G', 'y'], 'y': ['F', '∞'], 'd': ['∞', 'H'], 'm': ['π', 'K'], 'b': ['Ç', '—'], 'f': ['—', 'M'], 'p': ['J', '^'], 'g': ['I', 'Ω'], 'w': ['T', '‘'], 'v': ['N', 'Q'], 'k': ['D', 'µ']}), -484.92153959), (defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z', 'u'], 't': ['u', 'B'], 's': ['À', 'O'], 'i': ['R', '•'], 'r': ['•', '–'], 'h': ['–', 'X'], 'o': ['X', 'W'], 'c': ['V', '+'], 'u': ['≈', 'G'], 'l': ['G', 'y'], 'y': ['F', '∞'], 'd': ['∞', 'H'], 'm': ['π', 'K'], 'b': ['Ç', '—'], 'f': ['—', 'M'], 'p': ['J', '^'], 'g': ['I', 'Ω'], 'w': ['T', '‘'], 'k': ['N', 'Q'], 'v': ['Q', 'D'], 'j': ['µ']}), -492.45034995599997)]
	Working on symbol:  £
[(defaultdict(<class 'dict'>, {'e': ['∫', 'P'], 'n': ['/', 'Z'], 'a': ['Z', 'u'], 't': ['u', 'B'], 's': ['À', 'O']

In [79]:
reversed_mapping

[]

Notice that the default solution provides a very bad decipherment. Your job is to make it better!

## Grading

Ignore the following cells. They are for grading against the reference decipherment. Based on the clues provided in the decipherment homework description, you can easily find a reasonable reference text online for this cipher text.

In [ ]:
"""
ATTENTION!
For grading purposes only. Don't bundle with the assignment. 
Make sure '_ref.txt' is removed from the 'data' directory before publishing.
"""

def read_gold(gold_file):
    with open(gold_file) as f:
        gold = f.read()
    f.close()
    gold = list(gold.strip())
    return gold

def symbol_error_rate(dec, _gold):
    gold = read_gold(_gold)
    correct = 0
    if len(gold) == len(dec):
        for (d,g) in zip(dec, gold):
            if d==g:
                correct += 1
    wrong = len(gold)-correct
    error = wrong/len(gold)
    
    return error

In [ ]:
# gold decipherment
gold_file = "data/_ref.txt"
ser = symbol_error_rate(decipherment, gold_file)
print('Error: ', ser*100, 'Accuracy: ', (1-ser)*100)